Title Page / Header Cell 

Module: CT7201 Python Notebooks and Scripting 

Assignment Title: Earthquake Data Analysis Using Python Scripting (2023 Global Dataset) 

Student Names & IDs 

Date of submission 

Tutor Name 


Executive Summary (Short Overview) 

A single short paragraph that explains: 

What the project does 

What the dataset contains 

What analyses and models you will build 

The purpose of your Python scripting/OOP 

The key findings (a preview) 

Introduction 

Explain: 

Why we analyse earthquakes 

Why Python scripting is appropriate 

The importance of visualisation, functions, and clean coding 

A short explanation of what will be done in the notebook 

Dataset Description 

Cover: 

Source: USGS Earthquake Hazards Program 

Scope: Global events in 2023 

Number of records and variables 

Key fields (time, magnitude, depth, location, errors, network) 

Why this dataset is suitable for scripting and analysis 

Project Objectives 

Write them cleanly and academically: 

Load, clean, and prepare the earthquake dataset using Python scripting. 

Implement functions and modular code to automate analysis steps. 

Perform univariate, bivariate, and multivariate analysis. 

Produce clear and readable visualisations using matplotlib/seaborn. 

Implement a 3D visualisation using Python libraries. 

Build a simple machine learning model (classification or clustering). 

Demonstrate good programming practice, clarity, modularity, and documentation. 

 Methodology 

A clear step-by-step description of the workflow: 

Import libraries 

Load raw CSV 

Clean data and handle missing values 

Engineer additional features 

Perform exploratory analysis (EDA) 

Build visualisations 

Train and evaluate a simple ML model 

Interpret outputs 

Conclude findings 

Python Scripting & Functions Section 

CT7201 marks heavily focus on scripting. 

You MUST: 

✔ Create multiple custom Python functions: 

load_data() 

clean_data() 

engineer_features() 

plot_magnitude_distribution() 

plot_depth_boxplot() 

calculate_correlations() 

build_classifier() 

plot_3D_scatter() 

✔ Use: 

docstrings 

comments 

parameters 

return values 

✔ Avoid: 

long messy code cells 

repeating the same code 

In [ ]:
import setup_libs as libs

# Common aliases from the combined setup_libs module for convenience in the notebook
np = libs.np
pd = libs.pd
plt = libs.plt
sns = libs.sns
mpimg = libs.mpimg
Line2D = libs.Line2D

# Apply default plot style defined in setup_libs (safe to call)
try:
    libs.apply_default_plot_style()
except Exception:
    pass


In [ ]:
# load the raw csv file
#Dan

eq_df = libs.pd.read_csv('earthquake_dataset.csv')
eq_df.head()

In [ ]:
eq_df.describe()
#Dan

In [ ]:
eq_df.info()
#Dan

In [ ]:
# check maximum depth value
eq_df["depth"].max()

In [ ]:
#use histogram to visualize the depth
libs.plt.figure(figsize=(8,5))

libs.plt.hist(eq_df['depth'], bins=20, color='red')
libs.plt.xlim(0,700)
libs.plt.title('Visualizing the depth of Earthquakes in dataset')
libs.plt.xlabel('Depth(km)')
libs.plt.ylabel('Frequency')
libs.plt.show()

Shows all Earthquakes falling under the maximum of 700km

In [ ]:
#Plotting a distribution of Earthquake epicentres
libs.plt.figure(figsize=(10, 6))
libs.plt.scatter(eq_df['longitude'], eq_df['latitude'], s=1, alpha=0.5, c='darkblue')

libs.plt.xlim(-180, 180)
libs.plt.ylim(-90, 90)

libs.plt.title('Global Distribution of Earthquake Epicenters')
libs.plt.xlabel('Longitude (Degrees)')
libs.plt.ylabel('Latitude (Degrees)')
libs.plt.grid(True, linestyle='--', alpha=0.7)
libs.plt.savefig('epicentre_scatter.png')

libs.plt.show()
libs.plt.close()

![Global distribution of epicentres](epicentre_scatter.png)

Static render saved alongside this notebook for GitHub viewing.

Visualisation to show all resords falling between the longitude and latitude ranges.

In [ ]:
#finding the distribution of type of event
type_counts = eq_df['type'].value_counts()
print(type_counts)

In [ ]:
libs.plt.figure(figsize=(8,6))
ax = type_counts.plot(kind='bar', color = 'blue')
libs.plt.title('Distribution of Event types within dataset')
libs.plt.xlabel('Distribution of event types')
libs.plt.ylabel('Frequency')
libs.plt.xticks(rotation=45)


The dataset predominantly holds data for earthquakes globally.  However, some of the held data refers to other disasters such as mining explosions and ice quakes.  In order to focus this analysis on earthquake prediction, the other disasters will need to be filtered out as a part of the cleaning process.

In [ ]:
# show individual functions in these blocks
#load_data():


## 3.0 Data Cleaning

Reliable analysis begins with a dataset that is complete, consistent and physically meaningful. Earthquake data is particularly sensitive to measurement quality. Small errors in magnitude, depth, time or location can distort global patterns and weaken any statistical conclusions. For this reason, a structured and domain-appropriate cleaning pipeline was applied to the 2023 global earthquake dataset to ensure that every event used in this project is valid, accurate and suitable for analysis.

The dataset provided by the USGS is generally well structured, but still contains a number of issues that must be addressed before performing meaningful analysis. These include occasional fully duplicated rows, multiple versions of the same earthquake (with earlier records containing outdated measurements), a small number of values that fall outside physically realistic ranges and non earthquieke type explotions. The aim of the cleaning process is not to remove large portions of the data or introduce unnecessary filtering, but simply to correct errors and ensure that each earthquake record is scientifically reasonable.

The cleaning pipeline used in this project consists of the following key steps:

**1. Work on a copy of the data**  
A fresh copy of the dataset is created to protect the original raw data. This avoids accidental modifications and preserves the integrity of the input file.

**2. Remove exact duplicate records**  
Fully duplicated rows occasionally appear in exported or merged datasets. These provide no new information and would artificially inflate earthquake counts if left in place. All exact duplicates were removed.

**3. Remove duplicate earthquake identifiers, keeping only the most recent update**  
The `id` field uniquely identifies each earthquake. In some cases, the same event appears more than once because the USGS updates its magnitude, depth or location as new sensor data arrives. The dataset was sorted by the `updated` timestamp, ensuring that only the most recent and most accurate version of each event is retained.

It is important to note that repeated `id` values do not represent different “stages” of an earthquake. Earthquakes do not physically change in magnitude or depth over time. Multiple entries simply reflect measurement corrections by the seismic network. Therefore, retaining only the latest updated version ensures analytical accuracy.

**4. Convert time fields into proper datetime format**  
The `time` and `updated` columns were converted from text into real datetime objects. This step is required for all time-based analysis such as monthly grouping, temporal visualisation and event ordering.

**5. Ensure core numeric fields are numeric and remove missing essential values**
The core physical attributes of each earthquake latitude, longitude, depth and magnitude must be stored as numbers.These columns were converted to numeric values using errors="coerce", which safely transforms invalid entries (e.g., text or corrupted values) into NaN. Rows missing any of these essential fields were removed, as they cannot be displayed on maps or used in magnitude/depth analysis. The 2023 dataset is generally complete, so this step removed only a very small number of invalid rows.

**6. Remove rows missing essential physical attributes**  
Every earthquake must have a valid `time`, `latitude`, `longitude`, `depth` and `mag`. 
Without these, the event cannot be plotted, positioned geographically, or included in magnitude or depth analysis. Any rows missing these core fields were removed. The 2023 dataset is highly complete, so no events were lost at this stage.

**7. Apply geophysical validity checks**  
Basic physical constraints were enforced to ensure that all retained records represent real, possible earthquakes. These checks include:
- Latitude within −90 to 90 degrees  
- Longitude within −180 to 180 degrees  
- The deepest earthquake in the dataset occurred at 681 km, which is within the scientifically known maximum depth for natural earthquakes (∼700 km). For this reason, depth values were restricted to the range 0–700 km to remove impossible values while keeping all real deep-focus events
- Magnitude within the realistic range of 0 to 10
Any record falling outside these limits was removed, as such values reflect data errors rather than genuine seismic activity.

**8. Keep only real earthquakes (remove explosions and other event types)**
The dataset includes different types of seismic events, such as quarry blasts, mining explosions, ice quakes and testing activity.These are not natural earthquakes and would distort global patterns.Only rows where type == "earthquake" were retained.

Non-essential fields such as `gap`, `dmin`, `nst`, `horizontalError`, `magError` and 
`place` contain some missing values, but these do not prevent spatial or statistical analysis. Removing rows based on these optional metadata fields would unnecessarily reduce the dataset and bias the analysis toward only well-recorded earthquakes
These fields were therefore kept as they are.

In [ ]:
#Hasini
def clean_data(df):
    df = df.copy() # work on a copy to avoid modifying the original dataframe

    # Remove rows which are completely duplicated
    original_rows = len(df)
    df = df.drop_duplicates()
    print(f"Removed {original_rows - len(df)} exact duplicate rows.")

     # Convert 'time' and 'updated' columns to datetime
    df["time"] = pd.to_datetime(df["time"], errors="coerce")
    df["updated"] = pd.to_datetime(df["updated"], errors="coerce")

    # If the same earthquake ID appears more than once, keep the most recently added entry)
    rows_before_id_clean = len(df)
    df = (
        df.sort_values("updated") # oldest to newest
        .drop_duplicates(subset="id", keep="last")
    )
    print(f"Removed {rows_before_id_clean - len(df)} rows with duplicate earthquake based on IDs.")

    # Make sure the core numerical columns are actually numbers
    for col in ["latitude", "longitude", "depth", "mag"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # Drop rows missing essential earthquake information
    essential_cols = ["time", "latitude", "longitude", "depth", "mag"]
    rows_before_essential_clean = len(df)
    df = df.dropna(subset=essential_cols)
    print(f"Removed {rows_before_essential_clean - len(df)} rows with missing essential earthquake information in {essential_cols}.")

    # Keep only valid geographical ranges
    rows_before_ranges = len(df)

    valid_lat = df["latitude"].between(-90, 90)
    valid_lon = df["longitude"].between(-180, 180)
    valid_depth = df["depth"].between(0, 700)
    valid_mag = df["mag"].between(0, 10)

    df = df[valid_lat & valid_lon & valid_depth & valid_mag]
    print(f"Removed {rows_before_ranges - len(df)} rows with invalid geographical ranges.")

    #keep only type 'earthquake'
    if 'type' in df.columns:
        rows_before_type_clean = len(df)
        df = df[df["type"] == "earthquake"]
        print(f"Removed {rows_before_type_clean - len(df)} non earthquake events.")

    return df

In [ ]:
#Clean the dataset and check how its shape and structure change after cleaning.
#Hasini
print("Raw dataset shape:", eq_df.shape)

cleaned_eq_df = clean_data(eq_df)

print("Cleaned dataset shape:", cleaned_eq_df.shape)
cleaned_eq_df.info()
cleaned_eq_df.head()

Overall, The cleaning steps removed duplicates, outdated entries, invalid coordinates, impossible depths, magnitudes and non-earthquake events—while preserving all scientifically valid measurements. No rows were removed due to missing essential fields, confirming that the dataset is generally complete.

The final cleaned dataset is accurate, complete, and suitable for the exploratory analysis and machine-learning model presented later in this project. Importantly, the global structure of the data has been preserved, which is essential for analysing worldwide earthquake behaviour in 2023.

## 4.0 Feature Engineering

The cleaned 2023 USGS earthquake dataset (`cleaned_eq_df`) contains a physically consistent set of global earthquake events: valid latitude/longitude, non-negative depth, and realistic magnitude. The epicentre map from Section 3 showed events clustered along major tectonic plate boundaries, indicating the cleaning pipeline removed spurious measurements.

Clean data is not automatically analytically useful. Several fields are too raw for interpretation or modelling:
- `time` is a precise timestamp, not calendar components.
- Depth is in km, while seismologists reason in shallow/intermediate/deep.
- Magnitude is continuous; communication uses descriptors (moderate, strong, major).
- Latitude/longitude are exact but don’t directly compare regions (e.g., Americas vs Asia–Pacific).
- Data quality is spread across uncertainty fields (`gap`, `rms`, `depthError`, `magError`, `horizontalError`), not directly comparable.

Implementation guardrails:
- Leakage guard: If you later predict magnitude or depth, drop `mag_category`, `depth_category`, and `energy_log10_J` from training features to avoid label leakage.
- Missing-data policy: Timestamps are coerced to UTC; failed parses become `NaT`. Uncertainty fields may remain `NaN`; downstream models should impute or filter using `quality_score`.

We perform feature engineering: systematic transformations of raw measurements into higher-level variables useful to humans and algorithms. Engineered features are grouped into four concepts:
1) Temporal — when earthquakes occur  
2) Physical severity — how strong or deep they are  
3) Geographical context — where they occur in broad global terms  
4) Data quality — how reliable the measurements are  

All transformations live in `engineer_features()`. It takes `cleaned_eq_df` and returns `featured_eq_df` with the same rows but richer columns capturing temporal patterns, physical severity, spatial context, and measurement quality. This engineered view underpins Section 5 (exploration) and Section 6 (modelling).


In [ ]:
plt = libs.plt

fig, ax = plt.subplots(figsize=(8, 3.2))
ax.axis("off")

def box(x, n, text):
    ax.text(
        x, 0.5, f"{n}. {text}",
        ha="center", va="center",
        fontsize=10,
        bbox=dict(boxstyle="round,pad=0.6", fc="white", ec="black"),
    )

box(0.15, 1,
    r"$\bf{cleaned\_eq\_df}$" + "\n"
    "• Valid coordinates\n"
    "• Realistic depth & magnitude\n"
    "• Only type = 'earthquake'")
box(0.50, 2,
    r"$\bf{engineer\_features}$" + "\n"
    "• Temporal features\n"
    "• Physical severity\n"
    "• Geographic context\n"
    "• Data-quality indicators")
box(0.85, 3,
    r"$\bf{featured\_eq\_df}$" + "\n"
    "• Same number of rows\n"
    "• New engineered columns\n"
    "• ML-ready structure")


plt.title("Feature Engineering Workflow", fontsize=18, pad=15)
plt.tight_layout()
plt.show()


*Figure: cleaned catalogue → single `engineer_features()` pass → enriched `featured_eq_df` (same rows, richer columns: temporal, physical, spatial, quality). Re-run `engineer_features()` on any updated catalogue to regenerate an identical feature layer for comparability across years.*

### 4.1 Design of the `engineer_features()` Function

Principles:
1) Domain interpretability — features reflect seismology concepts (depth classes, magnitude labels, tectonic regions).  
2) Reproducibility/modularity — one function regenerates the full feature set consistently.

#### 4.1.1 Temporal Features — When Earthquakes Occur
- `year`, `month`, `month_name`
- `day`, `day_of_week`, `day_name`
- `hour`, `part_of_day` (night/morning/afternoon/evening)
- `is_weekend`
- `season`

#### 4.1.2 Physical Severity Features — How Strong or Deep
- `depth_category`: shallow (0–70 km), intermediate (70–300 km), deep (300–700 km)
- `mag_category`: minor, light, moderate, strong, major, great, massive
- `is_strong_quake`: mag ≥ 6.0
- `energy_log10_J`: log energy proxy (Gutenberg–Richter)

Depth bins follow the standard shallow/intermediate/deep convention. Magnitude bins mirror common communication labels. Small shifts to edges don’t materially change patterns; adjust if a modelling task needs different cut points.

#### 4.1.3 Geographical Context Features — Where Earthquakes Occur
- `abs_latitude`, `abs_longitude`
- `distance_from_equator_km`, `distance_from_prime_meridian_km`
- `hemisphere_NS`, `hemisphere_EW`
- `broad_region`: Americas, Europe–Africa, Asia–Pacific, Oceania

`broad_region` is a coarse tectonic grouping to reveal global patterns; a finer distance-to-plate-boundary feature is a natural future enhancement when plate geometries are available.

#### 4.1.4 Data-Quality Features — How Reliable the Measurements Are
- Boolean indicators (e.g., `has_depthError`)
- Min–max normalization of uncertainties
- Composite `quality_score` (1 = highest quality)

`quality_score` is an unweighted mean of normalized uncertainties; if some uncertainties matter more, use a weighted average or PCA-based composite as a refinement.

#### 4.1.5 Categorical Encodings
- Convenience ordinal codes `*_code` for categorical features.

Categorical codes (`*_code`) are convenience ordinals; for ML, one-hot or embed them, and scale continuous features before most models.


In [ ]:
np = libs.np
pd = libs.pd
from pandas.api.types import is_datetime64_any_dtype

def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Engineer temporal, physical, geographical, and data-quality features
    from the cleaned earthquake dataframe.
    """
    df = df.copy()

    # ===== 1. TEMPORAL FEATURES =====
    if not is_datetime64_any_dtype(df["time"]):
        df["time"] = pd.to_datetime(df["time"], errors="coerce", utc=True)

    dt = df["time"].dt
    df["year"] = dt.year
    df["month"] = dt.month
    df["month_name"] = dt.month_name()
    df["day"] = dt.day
    df["day_of_week"] = dt.dayofweek
    df["day_name"] = dt.day_name()
    df["hour"] = dt.hour
    df["is_weekend"] = df["day_of_week"].isin([5, 6])

    def hour_to_part(h):
        if h < 6:   return "night"
        if h < 12:  return "morning"
        if h < 18:  return "afternoon"
        return "evening"
    df["part_of_day"] = df["hour"].apply(hour_to_part)

    def month_to_season(m):
        if m in (12, 1, 2):  return "winter"
        if m in (3, 4, 5):   return "spring"
        if m in (6, 7, 8):   return "summer"
        return "autumn"
    df["season"] = df["month"].apply(month_to_season)

    # ===== 2. PHYSICAL SEVERITY =====
    df["depth_category"] = pd.cut(
        df["depth"],
        bins=[0, 70, 300, 700],
        labels=["shallow", "intermediate", "deep"],
        right=False
    )
    df["mag_category"] = pd.cut(
        df["mag"],
        bins=[0, 3, 4, 5, 6, 7, 8, 10],
        labels=["minor", "light", "moderate", "strong", "major", "great", "massive"],
        right=False
    )
    df["is_strong_quake"] = df["mag"] >= 6.0
    df["energy_log10_J"] = 1.5 * df["mag"] + 4.8

    # ===== 3. GEOGRAPHICAL CONTEXT =====
    df["abs_latitude"] = df["latitude"].abs()
    df["abs_longitude"] = df["longitude"].abs()
    df["distance_from_equator_km"] = df["abs_latitude"] * 111.0
    df["distance_from_prime_meridian_km"] = (
        df["abs_longitude"] * 111.0 * np.cos(np.deg2rad(df["latitude"]))
    )
    df["hemisphere_NS"] = np.where(df["latitude"] >= 0, "north", "south")
    df["hemisphere_EW"] = np.where(df["longitude"] >= 0, "east", "west")

    def classify_region(lon):
        if lon < -100: return "Americas_west"
        if lon < -30:  return "Americas_east_Atlantic"
        if lon < 60:   return "Europe_Africa"
        if lon < 150:  return "Asia_WestPacific"
        return "Pacific_Oceania"
    df["broad_region"] = df["longitude"].apply(classify_region)

    # ===== 4. DATA QUALITY =====
    for col in ["depthError", "magError", "horizontalError"]:
        if col in df.columns:
            df[f"has_{col}"] = df[col].notna()

    norm_cols = []
    for col in ["gap", "rms", "depthError", "magError", "horizontalError"]:
        if col in df.columns:
            mn, mx = df[col].min(), df[col].max()
            if pd.notna(mn) and pd.notna(mx) and mn != mx:
                norm_name = f"{col}_norm"
                df[norm_name] = (df[col] - mn) / (mx - mn)
                norm_cols.append(norm_name)
    if norm_cols:
        df["quality_score"] = 1 - df[norm_cols].mean(axis=1)

    # ===== 5. CATEGORICAL ENCODINGS =====
    cat_cols = [
        "depth_category", "mag_category", "hemisphere_NS",
        "hemisphere_EW", "broad_region", "part_of_day", "season"
    ]
    for c in cat_cols:
        if c in df.columns:
            df[f"{c}_code"] = df[c].astype("category").cat.codes

    return df


### 4.2 Engineered Feature Schema Summary


In [ ]:
# Engineered Feature Schema Summary
featured_eq_df = engineer_features(cleaned_eq_df)

# Ensure the engineered dataset exists
try:
    featured_eq_df
except NameError:
    raise NameError(
        "The variable 'featured_eq_df' is not defined.\n"
        "Run the feature-engineering step first:\n"
        "    featured_eq_df = engineer_features(cleaned_eq_df)"
    )

schema_groups = {
    "Temporal": ["year", "month", "season", "part_of_day", "is_weekend"],
    "Physical": ["depth_category", "mag_category", "is_strong_quake", "energy_log10_J"],
    "Spatial": ["hemisphere_NS", "hemisphere_EW", "broad_region", "distance_from_equator_km"],
    "Quality": ["quality_score", "gap_norm", "rms_norm", "has_depthError"],
    "ML Encodings": ["depth_category_code", "mag_category_code", "season_code", "part_of_day_code"],
}

print("Engineered Feature Schema Summary\n(showing only columns present in featured_eq_df)\n")

total_listed = set()
for group_name, columns in schema_groups.items():
    existing = [col for col in columns if col in featured_eq_df.columns]
    total_listed.update(existing)
    print(f"{group_name} ({len(existing)} features):")
    for col in existing:
        print(f"  • {col}")
    print()

print(f"Total engineered features listed: {len(total_listed)}")


The schema summary above provides a structured overview of all engineered
features added to the cleaned earthquake dataset. The features are grouped into
five conceptual domains Temporal, Physical, Spatial, Quality, and ML Encodings 
each serving a distinct analytical purpose.

The temporal group confirms that the original timestamp has been decomposed into
 interpretable components such as month, season, and part of day, enabling
fine-grained temporal analyses without relying on raw datetime strings. The
physical group shows the conversion of depth and magnitude into meaningful
severity descriptors, including depth categories and magnitude classes, which
align with standard seismological conventions. Spatial features provide
higher-level geographical context by transforming raw latitude and longitude
into hemispheres and broad tectonic regions. The quality indicators summarise
uncertainty and measurement reliability across several USGS-provided metrics,
allowing later analyses to differentiate between well-constrained and
poorly-constrained events.

The final group, ML Encodings, contains the integer representations of
categorical variables that are required for downstream modelling. These codes
are machine-readable placeholders and should not be interpreted as ordinal
quantities; for modelling, they will typically be one-hot encoded or embedded.

Overall, this schema demonstrates how the feature-engineering pipeline lifts the
raw USGS catalogue into a structured, analysis-ready dataset, with each feature
group supporting a different aspect of the exploratory and modelling workflows
that follow.


### 4.3 Feature Engineering Pipeline

In [ ]:
# Applying the Feature Engineering Pipeline

print("Applying feature engineering to cleaned_eq_df...\n")

# Record original column names
original_columns = list(cleaned_eq_df.columns)

# Apply the feature engineering function
featured_eq_df = engineer_features(cleaned_eq_df)

# Shape comparison
print("=== Dataset Shape Comparison ===")
print(f"Before feature engineering : {cleaned_eq_df.shape}")
print(f"After feature engineering  : {featured_eq_df.shape}")

# Identify new engineered features
new_cols = [c for c in featured_eq_df.columns if c not in original_columns]

print(f"\nNumber of engineered features added: {len(new_cols)}")
print("Sample of engineered feature names (first 20):")
for c in sorted(new_cols)[:20]:
    print(f"  • {c}")

# Display the first 5 rows of the engineered dataset
print("\nPreview of engineered dataframe:")
featured_eq_df.head()


The output above confirms that the feature engineering function has been applied
successfully to the cleaned USGS catalogue. The number of rows remains unchanged,
demonstrating that feature engineering enriches the dataset without altering the
underlying set of earthquake events. This is an essential property, as it ensures
that all subsequent analyses are based on the same physical observations that
were validated during the cleaning stage.

The increase in the number of columns reflects the addition of new engineered
features across the temporal, physical, spatial, and data-quality groups
introduced in Section 4.1. The summary shows how many new variables were created
and provides a sample of their names, illustrating how the raw fields have been
transformed into a richer and more analytically expressive representation.

The preview of `featured_eq_df` reveals the final structure of the enriched
dataset, allowing us to verify that the engineered features have been correctly
integrated. This includes checks such as: the presence of categorical descriptors
(e.g., `depth_category`, `mag_category`), temporal components (e.g., `month`,
`season`), spatial context variables (e.g., `broad_region`), and normalised
uncertainty metrics (e.g., `gap_norm`, `rms_norm`).

Overall, the pipeline output demonstrates that the dataset is now fully prepared
for exploratory analysis (Section 5) and for use as structured input in
machine-learning workflows (Section 6). The engineered features provide a more
interpretable, domain-aligned foundation for uncovering meaningful patterns in
global seismic activity.


### 4.4 Overview of Engineered Feature Groups

To validate that the engineered features behave sensibly and to provide an
at-a-glance summary of the enriched dataset, Figure 4.4 presents a four-panel
overview of the main feature groups. These panels correspond to:  
(a) temporal earthquake frequency,  
(b) magnitude severity categories,  
(c) spatial distribution across broad global regions, and  
(d) distribution of the composite measurement-quality score.

Together, these visualisations provide a compact diagnostic view of the
engineered feature space, confirming that the transformations applied in Section
4.1 produce interpretable and domain-consistent structures.



In [ ]:
plt = libs.plt
np = libs.np

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.ravel()
labels = ['(a)', '(b)', '(c)', '(d)']

# (a) Monthly frequency
order_months = [
    "January","February","March","April","May","June",
    "July","August","September","October","November","December"
]
month_counts = (
    featured_eq_df["month_name"]
    .value_counts()
    .reindex(order_months)
    .fillna(0)
)
x = np.arange(len(order_months))
axes[0].plot(x, month_counts.values, marker="o", linewidth=2)
axes[0].set_title("Monthly earthquake frequency", fontsize=12)
axes[0].set_xticks(x)
axes[0].set_xticklabels(order_months, rotation=45, ha="right")
axes[0].set_ylabel("Number of earthquakes")
axes[0].grid(axis="y", linestyle="--", alpha=0.3)
axes[0].text(-0.12, 1.05, labels[0], transform=axes[0].transAxes,
             fontsize=13, fontweight="bold")

# (b) Magnitude categories
mag_counts = (
    featured_eq_df["mag_category"]
    .value_counts()
    .sort_index()
)
y_pos = np.arange(len(mag_counts))
axes[1].barh(y_pos, mag_counts.values, color="#cfe2ff", edgecolor="#2f2f2f")
axes[1].set_yticks(y_pos)
axes[1].set_yticklabels(mag_counts.index.astype(str))
axes[1].invert_yaxis()
axes[1].set_title("Magnitude category distribution", fontsize=12)
axes[1].set_xlabel("Number of earthquakes")
axes[1].grid(axis="x", linestyle="--", alpha=0.3)
axes[1].text(-0.12, 1.05, labels[1], transform=axes[1].transAxes,
             fontsize=13, fontweight="bold")
for i, v in enumerate(mag_counts.values):
    axes[1].text(v + mag_counts.values.max()*0.01, i, str(v),
                 va="center", fontsize=9)

# (c) Broad regions
region_counts = (
    featured_eq_df["broad_region"]
    .value_counts()
    .sort_index()
)
x2 = np.arange(len(region_counts))
axes[2].bar(x2, region_counts.values, color="#ffe5d9", edgecolor="#2f2f2f")
axes[2].set_xticks(x2)
axes[2].set_xticklabels(region_counts.index.astype(str), rotation=20, ha="right")
axes[2].set_title("Earthquake counts by broad region", fontsize=12)
axes[2].set_ylabel("Number of earthquakes")
axes[2].grid(axis="y", linestyle="--", alpha=0.3)
axes[2].text(-0.12, 1.05, labels[2], transform=axes[2].transAxes,
             fontsize=13, fontweight="bold")

# (d) Quality score
ax = axes[3]
if "quality_score" in featured_eq_df.columns:
    qs = featured_eq_df["quality_score"].dropna()
    ax.hist(qs, bins=20, edgecolor="black", alpha=0.7, color="#d1e7dd")
    median_qs = qs.median()
    ax.axvline(median_qs, color="red", linestyle="--", linewidth=1.5,
               label=f"Median = {median_qs:.2f}")
    ax.set_title("Quality score distribution", fontsize=12)
    ax.set_xlabel("Quality score (1 = best)")
    ax.set_ylabel("Frequency")
    ax.grid(axis="y", linestyle="--", alpha=0.3)
    ax.legend(fontsize=9)
else:
    ax.text(0.5, 0.5, "quality_score not available",
            ha="center", va="center", fontsize=11)
    ax.axis("off")
ax.text(-0.12, 1.05, labels[3], transform=ax.transAxes,
        fontsize=13, fontweight="bold")

plt.tight_layout()
plt.show()


**Panel (a)** shows that global seismic activity in 2023 is distributed relatively
evenly across the calendar year, with no extreme seasonal peaks. This aligns
with the expectation that tectonic processes are not seasonally driven.

**Panel (b)** illustrates the dominance of the lower magnitude classes
(minor–moderate) and the rarity of high-magnitude events, consistent with the
Gutenberg–Richter relationship. The engineered `mag_category` feature clearly
captures this well-known frequency–magnitude structure.

**Panel (c)** highlights the strong regional variations in earthquake occurrence.
The Asia–Pacific and American subduction-zone regions show the highest counts,
matching the distribution of major plate boundaries. This validates the use of
the `broad_region` engineered feature.

**Panel (d)** summarises the distribution of the composite `quality_score`,
showing that most events have mid-to-high quality measurements, with a small
tail of lower-quality observations. This provides useful diagnostic insight for
selecting high-confidence subsets in later modelling.

Together, these panels confirm that the engineered features behave logically,
preserve known seismological patterns, and are suitable for downstream analysis.



### 4.5 Additional Feature-Level Visualisations

To complement the high-level overview in Figure 4.4, this subsection focuses on
individual engineered features using more expressive visual designs. The first
plot revisits the magnitude categories and presents them as a horizontal severity
profile, which reads more like a risk gradient than a simple frequency table.



#### 4.5.1 Magnitude severity profile

Figure 4.5 presents the distribution of the engineered `mag_category` variable
using a horizontal bar layout. Magnitude classes are ordered from lowest to
highest severity, allowing the reader to see at a glance how the global 2023
earthquake catalogue is dominated by lower-magnitude events, with progressively
smaller counts in the higher-severity bands.


In [ ]:
plt = libs.plt
np = libs.np

mag_counts = (
    featured_eq_df["mag_category"]
    .value_counts()
    .sort_index()
)

colors = plt.cm.plasma(np.linspace(0.25, 0.95, len(mag_counts)))

plt.figure(figsize=(10, 5))
plt.barh(
    mag_counts.index.astype(str),
    mag_counts.values,
    color=colors,
    edgecolor="#2f2f2f"
)

plt.title("Magnitude severity profile (enhanced view)", fontsize=13)
plt.xlabel("Number of earthquakes")
plt.ylabel("Magnitude category")
plt.grid(axis="x", linestyle="--", alpha=0.4)

# Add counts at bar ends
for i, v in enumerate(mag_counts.values):
    plt.text(
        v + max(mag_counts.values) * 0.01,
        i,
        str(v),
        va="center",
        fontsize=9
    )

plt.tight_layout()
plt.show()


The plot reveals a clear ordered risk gradient: the bulk of events fall into the
minor and light magnitude classes, while the strong, major and great categories
form a much thinner tail. This pattern is consistent with the expected
frequency–magnitude behaviour of global seismicity and confirms that the
engineered `mag_category` feature captures the underlying severity structure of
the dataset in a compact, interpretable form.


#### 4.5.2 Magnitude distribution by depth category

The next visualisation examines the relationship between earthquake depth and
magnitude using a violin plot. Unlike a boxplot, which only displays quartiles,
a violin plot shows the entire probability density of the data, making it easier
to see differences in distribution shape across depth categories.


In [ ]:
plt.figure(figsize=(9, 5))
data = [featured_eq_df[featured_eq_df["depth_category"] == cat]["mag"].dropna()
    for cat in ["shallow", "intermediate", "deep"]]

plt.violinplot(data, showmeans=True, showextrema=True)
plt.title("Magnitude distribution by depth category", fontsize=13)
plt.xlabel("Depth category")
plt.ylabel("Magnitude")
plt.xticks([1, 2, 3], ["Shallow", "Intermediate", "Deep"])
plt.grid(axis="y", linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()


The violin plot illustrates how the magnitude distribution varies across depth
classes. Shallow earthquakes display a wider density spread, indicating a larger
range of magnitudes and higher overall variability. Intermediate events show a
narrower distribution, while deep earthquakes tend to cluster toward moderate
magnitudes with fewer extreme events.

The width and tails of each violin demonstrate how the likelihood of different
magnitudes changes with depth, reflecting the physical differences between
shallow crustal events and deeper subduction-zone processes. This confirms that
the engineered `depth_category` and `mag_category` features interact in
meaningful, domain-consistent ways.


#### 4.5.3 Global epicentre distribution with magnitude–depth

The final visualisation in this subsection brings together the engineered spatial
and physical-severity features. Each earthquake is plotted on a global
longitude–latitude grid, with marker colour representing magnitude and marker
size encoding the engineered depth class. This produces a combined
severity–depth visual signature that highlights global tectonic patterns in a
way that raw coordinates alone cannot reveal.


In [ ]:
plt = libs.plt
mpimg = libs.mpimg
np = libs.np
pd = libs.pd
Line2D = libs.Line2D


# 1. LOAD AND CROP WORLD MAP (SOFTER, HIGHER CONTRAST)

img = mpimg.imread("world_map.png")

# Crop black borders
rgb = img[:, :, :3]
brightness = rgb.mean(axis=2)
mask = brightness > 0.02
rows = np.where(mask.any(axis=1))[0]
cols = np.where(mask.any(axis=0))[0]
cropped_img = img[rows[0]:rows[-1], cols[0]:cols[-1]]

# Slight contrast adjustment (not too dark)
cropped_img = np.clip(cropped_img * 0.9, 0, 1)


# 2. IMPORT TECTONIC PLATE BOUNDARIES

plates = pd.read_csv("plate_boundaries.csv")


# 3. Marker sizes by depth category

size_map = {"shallow": 32, "intermediate": 20, "deep": 12}
marker_sizes = (
    featured_eq_df["depth_category"]
    .map(size_map)
    .astype(float)
    .fillna(14.0)
)


# 4. Create final figure

fig, ax = plt.subplots(figsize=(14, 6))

# Background world map
ax.imshow(
    cropped_img,
    extent=[-180, 180, -90, 90],
    aspect="auto",
    alpha=0.75,      # slightly more transparent so data stands out
    zorder=0
)

# Earthquake points
sc = ax.scatter(
    featured_eq_df["longitude"],
    featured_eq_df["latitude"],
    c=featured_eq_df["mag"],
    s=marker_sizes,
    alpha=0.8,
    cmap="viridis",
    edgecolors="none",
    zorder=2
)


# 5. Tectonic plate boundaries overlay

ax.plot(
    plates["lon"], plates["lat"],
    color="red", linewidth=1.3, linestyle="--",
    alpha=0.8, zorder=3
)


# 6. Hotspot annotations

hotspots = {
    "Japan": (140, 38),
    "Chile": (-72, -30),
    "Turkey": (35, 39),
    "Tonga": (-175, -20),    # already in -180..180
    "California": (-122, 37)
}

for name, (x, y) in hotspots.items():
    ax.scatter(x, y, s=60, c="red", edgecolors="white", linewidth=0.7, zorder=4)
    ax.text(
        x + 4, y + 4,
        name,
        fontsize=9,
        color="red",
        fontweight="bold",
        zorder=5
    )


# 7. Annotate key tectonic provinces (black labels)
tectonic_labels = {
    "Andes": (-70, -15),
    "Cascadia": (-125, 45),
    "Mediterranean Belt": (20, 38),
    "Himalayas": (80, 30),
    "East Africa Rift": (35, 2),
    "Indonesia Arc": (120, -2),
    "Japan Trench": (145, 38),
    "Tonga Arc": (-175, -18),
}

for name, (lon, lat) in tectonic_labels.items():
    ax.text(
        lon, lat, name,
        fontsize=9, color="black",
        ha="center", va="center",
        alpha=1, zorder=4,
    )




# 8. Axis, ticks, grids

ax.set_title(
    "Global earthquake epicentres (2023)\n"
    "with tectonic boundaries and hotspot annotations",
    fontsize=20
)
ax.set_xlabel("Longitude (°)")
ax.set_ylabel("Latitude (°)")

ax.set_xlim(-180, 180)
ax.set_ylim(-90, 90)
ax.set_xticks(np.arange(-180, 181, 60))
ax.set_yticks(np.arange(-90, 91, 30))
ax.grid(True, linestyle="--", alpha=0.25)


# 9. Colourbar for magnitude

cbar = plt.colorbar(sc, ax=ax)
cbar.set_label("Magnitude")


# 10. Legends (depth + boundaries)

depth_legend_elements = [
    Line2D(
        [0], [0],
        marker='o', linestyle='',
        markersize=np.sqrt(size_map["shallow"]),
        label='Shallow (0–70 km)',
        markerfacecolor='grey', alpha=0.8
    ),
    Line2D(
        [0], [0],
        marker='o', linestyle='',
        markersize=np.sqrt(size_map["intermediate"]),
        label='Intermediate (70–300 km)',
        markerfacecolor='grey', alpha=0.8
    ),
    Line2D([0], [0], marker='o', linestyle='',
        markersize=np.sqrt(size_map["deep"]),
        label='Deep (300–700 km)',
        markerfacecolor='grey', alpha=0.8
    ),
]

# First legend: depth categories
depth_legend = ax.legend(
    handles=depth_legend_elements,
    title="Depth category",
    loc="lower left",
    fontsize=8,
    title_fontsize=9,
    frameon=True
)
ax.add_artist(depth_legend)

# Second legend: tectonic boundaries + hotspots
boundary_hotspot_legend = ax.legend(
    handles=[
        Line2D([0], [0], color="red", linestyle="--", linewidth=1.3,
               label="Tectonic plate boundaries"),
        Line2D([0], [0], marker="o", linestyle="", markersize=6,
               markerfacecolor="red", markeredgecolor="white",
               label="Annotated hotspot")
    ],
    loc="lower right",
    fontsize=8,
    frameon=True
)

plt.tight_layout()
plt.show()


## Interpretation of Global Earthquake Epicentres

The enhanced global epicentre map presents a geophysically coherent distribution of earthquake activity, with events aligning precisely along the major tectonic plate interfaces. The strongest and most persistent seismic zones—such as the **Pacific Ring of Fire**, the **Japan–Kurile arc**, the **Tonga–Kermadec subduction system**, the **Indonesian convergence zone**, and the **Peru–Chile trench**—appear as continuous, high-density belts of epicentres. These alignments coincide exactly with expected plate boundaries, confirming both the geographical accuracy of the cleaned dataset and the correctness of the engineered spatial variables.

The visual encoding integrates two fundamental physical dimensions of earthquakes:  
- **magnitude**, represented by a perceptually uniform colour scale, and  
- **focal depth**, expressed through marker size.  

This dual encoding reveals important physical structure. **Shallow earthquakes** (0–70 km), shown as larger markers, dominate global seismicity and cluster densely along subduction interfaces, where interplate friction generates frequent, high-magnitude thrust events. **Intermediate-depth earthquakes** (70–300 km) trace well-defined Wadati–Benioff zones beneath Japan, Tonga, Indonesia and the Andean margin, illustrating the geometry of subducting slabs. **Deep-focus earthquakes** (300–700 km), while comparatively rare, appear exclusively within these subduction corridors—consistent with established geodynamic models of cold, sinking lithosphere.

The inclusion of annotated hotspots (Japan, Chile, Turkey, Tonga, California) further emphasises regions of chronic stress accumulation and complex fault interaction. Complementary regional labels—such as the **Mediterranean Belt**, **Himalayas**, **Cascadia**, and the **East African Rift**—demonstrate that the engineered geographic variables (e.g. `broad_region`, `hemisphere_NS`, `distance_from_equator_km`) correctly segment the dataset into meaningful tectonic provinces.

Overall, the figure provides strong validation of the feature-engineering pipeline. The enriched dataset preserves the expected structural patterns of global seismicity, while the engineered attributes (depth categories, magnitude classes, distance measures, and region codes) make previously implicit geophysical relationships more observable. The map therefore functions as both a diagnostic check of dataset integrity and a substantive foundation for the exploratory and modelling work presented in subsequent sections.


### 4.6 Summary and Reflection

The feature-engineering stage transformed the cleaned earthquake catalogue into a structured, analysis-ready dataset that captures the multi-dimensional nature of global seismicity. The `engineer_features()` function integrates temporal descriptors (e.g. month, season, diurnal cycle), physical severity indicators (magnitude class, depth category, energy proxy), spatial context (hemispheres, distance measures, broad tectonic regions), and data-quality metrics (normalised uncertainty measures and composite quality scores). Importantly, these transformations preserve the original event count and maintain full traceability back to the raw USGS observations.

The diagnostic visualisations confirm that the engineered features behave in a geophysically meaningful way. Temporal variables reveal realistic seasonal and monthly variations in global seismic reporting. Depth and magnitude classes highlight the dominance of shallow crustal earthquakes and the physical structure of subduction-driven Wadati–Benioff zones. Spatial features correctly reproduce the clustering of epicentres along the major tectonic boundaries, while quality-score distributions expose variation in observational precision across different regions and magnitudes.

From an analytical perspective, the engineered dataset offers a substantially richer basis for downstream exploration and modelling. The introduction of interpretable categories improves descriptive analysis, while the numeric encodings (e.g. `*_code`) provide a machine-learning–friendly representation. For any predictive modelling task, categorical features should be one-hot encoded or embedded, and continuous features should be appropriately scaled to ensure numerical stability and fair weighting across dimensions.

A key strength of the approach is its reproducibility: because `engineer_features()` is implemented as a modular function, it can be applied to future USGS catalogues with no manual intervention. This guarantees that new datasets will inherit the same engineered structure, ensuring consistent analysis pipelines over time. Overall, the feature-engineering process successfully bridges raw seismic measurements and structured analytical insight, forming the foundation for the exploratory analysis and modelling undertaken in the subsequent sections.


- 5. Object-Oriented Design & Implementation  
5.1 EarthquakeDataset Class  
5.2 EarthquakeVisualizer Class  
5.3 EarthquakeModel Class (optional)  

- 6. Data Preparation (Using the OOP Pipeline)  
6.1 Loading the Dataset  
6.2 Cleaning the Dataset  
6.3 Feature Engineering  
6.4 Final Processed Dataset  

In [ ]:
import importlib
import sys

# Remove cached modules to force fresh import
if 'setup_libs' in sys.modules:
    del sys.modules['setup_libs']

import setup_libs as libs

print("Checking setup_libs imports (post-reload):")
print(f"libs.plt = {libs.plt}")
print(f"libs.sns = {libs.sns}")
print(f"libs.HAS_SEABORN = {libs.HAS_SEABORN}")
print(f"\nAvailability: {libs.availability()}")

- 7. Univariate Analysis (EDA – Part 1)  #DAN   
7.1 Magnitude Distribution (KDE)  
7.2 Depth Distribution (Histogram & Boxplot)  
7.3 Station Count (nst / magNst)  
7.4 Distance to Nearest Station (dmin)  
7.5 Temporal Frequency (Monthly / Daily Counts)  

In [ ]:
#Magnitude distribution
conditions =[
    (eq_df['depth'] >= 0) & (eq_df['depth'] <= 70),
    (eq_df['depth'] > 70) & (eq_df['depth'] <= 300),
    (eq_df['depth'] > 300) & (eq_df['depth'] <= 700)
]

depth_ranges = ['Shallow(0-70 km)', 'Intermediate (70-300 km)', 'Deep-focus (300-700 km)']
eq_df['depth_category'] = np.select(conditions, depth_ranges, default = 'invalid')

plt = libs.plt
sns = libs.sns

plt.figure(figsize=(10,8))
for depth_range in depth_ranges:
    subset=eq_df[eq_df['depth_category'] == depth_range]
    sns.kdeplot(subset['mag'],
                label=depth_range,
                fill=True,
                alpha = 0.5)

plt.title('Magnitude distribution by depth category')
plt.xlabel('Magnitude')
plt.ylabel('Density')
plt.xlim(0,10)
plt.legend(title='Earthquake depth category')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
#Depth Distribution (Histogram & Boxplot)

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
fig.suptitle('Earthquake Depth Distribution Analysis (0-700 km)', fontsize=16)

axes[0].hist(eq_df['depth'], bins=50, color='darkred', edgecolor='black')
axes[0].set_title('Histogram: Frequency of Depth')
axes[0].set_xlabel('Depth (km)')
axes[0].set_ylabel('Number of Earthquakes (Frequency)')
axes[0].set_xlim(0, 700)
axes[0].grid(axis='y', linestyle='--', alpha=0.7)


axes[1].boxplot(eq_df['depth'], vert=True, patch_artist=True, 
                boxprops=dict(facecolor='skyblue', color='darkblue'),
                medianprops=dict(color='red', linewidth=2))

axes[1].set_title('Box Plot: Summary of Depth')
axes[1].set_ylabel('Depth (km)')
axes[1].set_ylim(0, 700)
axes[1].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
#Station count

nst_data = eq_df['nst'].dropna()
magNst_data = eq_df['magNst'].dropna()

plt = libs.plt
sns = libs.sns

fig, axes = plt.subplots(1, 2, figsize=(16, 8))
fig.suptitle('Distribution Comparison: Station Count (nst vs. magNst)', fontsize=18)

max_limit = 100 
bins_count = 60

sns.histplot(nst_data, bins=bins_count, ax=axes[0], color='teal', edgecolor='black', kde=True)
axes[0].set_title('nst: Number of Stations Used (Overall)')
axes[0].set_xlabel('Station Count')
axes[0].set_ylabel('Frequency')
axes[0].set_xlim(0, max_limit)
axes[0].grid(axis='y', linestyle='--', alpha=0.6)

sns.histplot(magNst_data, bins=bins_count, ax=axes[1], color='darkorange', edgecolor='black', kde=True)
axes[1].set_title('magNst: Number of Stations Used for Magnitude')
axes[1].set_xlabel('Station Count')
axes[1].set_ylabel('Frequency')
axes[1].set_xlim(0, max_limit)
axes[1].grid(axis='y', linestyle='--', alpha=0.6)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
#Distance to the nearest station

dmin_data = eq_df['dmin'].dropna()

plt = libs.plt
sns = libs.sns

plt.figure(figsize=(8, 6))

sns.histplot(dmin_data, bins=50, color='mediumblue', edgecolor='black', kde=True)

plt.xlim(0, 5) 

plt.title('Distribution of Dmin (Distance to Nearest Station)')
plt.xlabel('Dmin (Distance in Degrees)')
plt.ylabel('Frequency')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
#Temporal Frequency (Monthly / Daily Counts)

eq_df['time'] = pd.to_datetime(eq_df['time'])
eq_df['day_of_week'] = eq_df['time'].dt.dayofweek
eq_df['hour_of_day'] = eq_df['time'].dt.hour

plt = libs.plt
sns = libs.sns

fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Earthquake Frequency by Temporal Period', fontsize=16)

day_labels = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
day_counts = eq_df['day_of_week'].value_counts().sort_index()

sns.barplot(x=day_counts.index, y=day_counts.values, ax=axes[0], color='lightcoral')
axes[0].set_title('Frequency by Day of the Week (GMT/UTC)')
axes[0].set_xlabel('Day of the Week')
axes[0].set_ylabel('Total Number of Earthquakes')
axes[0].set_xticks(range(7))
axes[0].set_xticklabels(day_labels)
axes[0].grid(axis='y', linestyle='--', alpha=0.6)

hour_counts = eq_df['hour_of_day'].value_counts().sort_index()
all_hours = pd.Series(0, index=range(24)).add(hour_counts, fill_value=0)

sns.lineplot(x=all_hours.index, y=all_hours.values, ax=axes[1], marker='o', color='darkgreen')
axes[1].set_title('Frequency by Hour)')
axes[1].set_xlabel('Hour')
axes[1].set_ylabel('Total Number of Earthquakes')
axes[1].set_xticks(range(0, 24, 2))
axes[1].grid(True, linestyle='--', alpha=0.6)


plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

### 8. Multivariate / Spatial / Quality EDA (Part 2)
This section interlinks magnitude, depth, spatial location, and the engineered `quality_score` to understand whether severe events align with specific depth ranges or regions and whether instrumentation uncertainty should influence downstream modelling.

Each subsection reuses `featured_eq_df` (cleaned + engineered features) and is safe to re-run independently:
- **8.1 Magnitude vs Depth** - correlate physical severity with focal depth classes.
- **8.2 Global Spatial Distribution** - world epicentre map plus regional share table.
- **8.3 Regional Seismicity Comparisons** - compare severity/depth mixes by region.
- **8.4 Quality Score & Uncertainty** - surface low-quality measurements before modelling.


#### 8.1 Magnitude vs Depth
Deep-focus earthquakes produce different shaking profiles from shallow crustal events. The scatter + distribution combo below highlights whether high magnitudes cluster at particular depth ranges.


In [ ]:
sns = libs.sns
pd = libs.pd
np = libs.np
plt = libs.plt

depth_mag_df = featured_eq_df[["depth", "mag", "depth_category", "mag_category"]].dropna(subset=["depth", "mag"]).copy()
if depth_mag_df.empty:
    raise ValueError("No depth/magnitude samples available for Section 8.1.")

sample_limit = 8000
plot_df = depth_mag_df.sample(sample_limit, random_state=42) if len(depth_mag_df) > sample_limit else depth_mag_df
corr_value = float(depth_mag_df["mag"].corr(depth_mag_df["depth"]))
depth_mag_df["depth_bin"] = pd.cut(
    depth_mag_df["depth"],
    bins=[0, 50, 100, 200, 400, 700],
    include_lowest=True,
    duplicates="drop"
)

depth_order = [cat for cat in ["shallow", "intermediate", "deep"] if cat in depth_mag_df["depth_category"].dropna().unique()]
depth_order = depth_order if depth_order else None
fig, axes = plt.subplots(1, 2, figsize=(15, 6), gridspec_kw={"width_ratios": [2.25, 1]})
sns.scatterplot(
    data=plot_df,
    x="depth",
    y="mag",
    hue="depth_category",
    size="mag",
    sizes=(20, 140),
    palette="viridis",
    alpha=0.65,
    ax=axes[0]
)
axes[0].set_title("Magnitude vs Depth (sampled)")
axes[0].set_xlabel("Depth (km)")
axes[0].set_ylabel("Magnitude (Mw)")
axes[0].grid(True, linestyle="--", alpha=0.35)
axes[0].text(
    0.02,
    0.05,
    f"Pearson r = {corr_value:.2f}",
    transform=axes[0].transAxes,
    bbox={"facecolor": "white", "alpha": 0.7, "edgecolor": "gray"}
)

sns.boxenplot(
    data=depth_mag_df,
    x="depth_category",
    y="mag",
    hue="depth_category",
    order=depth_order,
    palette="viridis",
    dodge=False,
    ax=axes[1]
)
legend = axes[1].get_legend()
if legend is not None:
    legend.remove()
axes[1].set_title("Magnitude distribution by depth class")
axes[1].set_xlabel("Depth category")
axes[1].set_ylabel("Magnitude (Mw)")
axes[1].grid(True, axis="y", linestyle="--", alpha=0.35)

plt.tight_layout()

depth_summary = (
    depth_mag_df.dropna(subset=["depth_bin"])
    .groupby("depth_bin", observed=False)
    .agg(
        events=("mag", "size"),
        median_mag=("mag", "median"),
        pct_strong=("mag", lambda s: 100 * (s >= 6.0).mean())
    )
)
depth_summary["events_pct"] = depth_summary["events"] / len(depth_mag_df) * 100
depth_summary = depth_summary[["events", "events_pct", "median_mag", "pct_strong"]]
depth_summary = depth_summary.round({"events": 0, "events_pct": 1, "median_mag": 2, "pct_strong": 1})
depth_summary


- **Correlation panel** - Pearson `r` quantifies coupling between depth and magnitude; |r| above ~0.3 suggests physical linkage worth modelling.
- **Distribution panel** - boxenplot tails reveal whether deep foci host disproportionately large events.
- **Depth-bin table** - cite `events_pct` and `pct_strong` when explaining which depth slices shoulder Mw ≥ 6 activity.


#### 8.2 Global Spatial Distribution (Epicentre Map)
An interactive Plotly globe (with a Matplotlib fallback) shows epicentres scaled by magnitude, while the accompanying table quantifies each macro region's share of global and strong events.

- **Interactive map (HTML)**: open the saved map in your browser via [raw link](https://raw.githubusercontent.com/DanPenso/earthquake-data-analysis/main/earthquake-data-analysis/epicentre_map_section8_2.html). If GitHub shows plain text, use "Save link as..." then open the file locally for full interactivity.
- **Static preview (PNG)**: see the PNG below; regenerate via `python export_epicentre_map.py` if you rerun the analysis.


In [ ]:
px = libs.px
plt = libs.plt
pd = libs.pd
np = libs.np

map_cols = ["latitude", "longitude", "mag", "mag_category", "broad_region", "depth", "place"]
available_cols = [c for c in map_cols if c in featured_eq_df.columns]
map_df = featured_eq_df[available_cols].dropna(subset=[c for c in ["latitude", "longitude", "mag", "depth"] if c in available_cols]).copy()
if map_df.empty:
    raise ValueError("No latitude/longitude samples available for Section 8.2.")

map_df["mag_size"] = np.clip(map_df["mag"], 3, None)
map_df["is_strong"] = map_df["mag"] >= 6.0
map_df["hover_label"] = (
    map_df["place"].fillna("Unknown location")
    + " | Mw " + map_df["mag"].round(1).astype(str)
    + " | depth " + map_df["depth"].round(0).astype(int).astype(str) + " km"
)

if px is not None:
    fig = px.scatter_geo(
        map_df,
        lat="latitude",
        lon="longitude",
        color="broad_region",
        size="mag_size",
        size_max=18,
        hover_name="hover_label",
        hover_data={"mag":":.1f", "depth":":.0f", "mag_category":True, "broad_region":True},
        projection="natural earth",
        template="plotly_white",
        title="Global epicentre distribution (bubble size ~ magnitude)"
    )
    fig.update_layout(legend_title_text="Broad region")
    try:
        fig.show()
    except ValueError as exc:
        if "nbformat" in str(exc):
            import plotly.io as pio
            pio.renderers.default = "browser"  # fallback when nbformat is missing
            fig.show()
        else:
            raise
else:
    fig, ax = plt.subplots(figsize=(14, 7))
    scatter = ax.scatter(
        map_df["longitude"],
        map_df["latitude"],
        c=map_df["mag"],
        cmap="plasma",
        s=np.square(map_df["mag_size"]) * 1.5,
        alpha=0.6,
        linewidths=0.15,
        edgecolor="black"
    )
    ax.set_title("Global epicentre distribution (bubble size ~ magnitude)")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_xlim(-180, 180)
    ax.set_ylim(-90, 90)
    ax.grid(True, linestyle="--", alpha=0.3)
    plt.colorbar(scatter, ax=ax, label="Magnitude (Mw)")

region_summary = (
    map_df.assign(strong_pct=map_df["is_strong"].astype(float))
    .groupby("broad_region")
    .agg(
        events=("mag", "size"),
        pct_global=("mag", lambda s: 100 * len(s) / len(map_df)),
        pct_strong=("strong_pct", "mean"),
        median_depth=("depth", "median")
    )
    .sort_values("events", ascending=False)
)
region_summary["pct_strong"] = region_summary["pct_strong"] * 100
region_summary = region_summary.round({"pct_global": 1, "pct_strong": 1, "median_depth": 0})
region_summary

# Optional: persist Section 8.2 outputs (set True to write files)
export_epicentre_outputs = False
if export_epicentre_outputs:
    region_summary.to_csv("region_summary_section8_2.csv")
    if px is not None:
        fig.write_html("epicentre_map_section8_2.html", include_plotlyjs="cdn")
        try:
            fig.write_image("epicentre_map_section8_2.png", scale=2)
        except ValueError as exc:
            if "kaleido" in str(exc).lower():
                print("Plotly static export skipped (kaleido not installed).")
            else:
                raise
    else:
        fig.savefig("epicentre_map_section8_2.png", dpi=300, bbox_inches="tight")



![Section 8.2 epicentre map](epicentre_map_section8_2.png)

Static PNG generated by the export hook; use the [HTML map](https://raw.githubusercontent.com/DanPenso/earthquake-data-analysis/main/earthquake-data-analysis/epicentre_map_section8_2.html) for pan/zoom/hover and `region_summary_section8_2.csv` for the table.


- **Geospatial bubble map** - hover any bubble to retrieve location, magnitude, and depth. Clusters hugging the Pacific rim visually validate plate-boundary hypotheses.
- **Regional share table** - `pct_global` shows contribution to the catalogue, while `pct_strong` highlights where Mw ≥ 6 energy release concentrates.


#### 8.3 Regional Seismicity Comparisons
Quantify how the magnitude and depth mix differs by `broad_region`. The combined bar/line view emphasises volume vs. major-quake share, while the heatmap captures depth proportions.


In [ ]:
sns = libs.sns
pd = libs.pd
plt = libs.plt

region_cols = ["broad_region", "mag", "depth", "depth_category"]
region_cols = [c for c in region_cols if c in featured_eq_df.columns]
region_df = featured_eq_df[region_cols].dropna(subset=["broad_region", "mag"]).copy()
if region_df.empty:
    raise ValueError("No regional samples available for Section 8.3.")

region_df["is_major"] = region_df["mag"] >= 7.0

region_summary = (
    region_df.groupby("broad_region")
    .agg(
        events=("mag", "size"),
        median_mag=("mag", "median"),
        iqr_mag=("mag", lambda s: s.quantile(0.75) - s.quantile(0.25)),
        pct_major=("is_major", "mean"),
        median_depth=("depth", "median")
    )
    .sort_values("events", ascending=False)
)
region_summary["pct_major"] = region_summary["pct_major"] * 100
region_plot = region_summary.reset_index()

depth_mix = pd.crosstab(region_df["broad_region"], region_df["depth_category"], normalize="index") * 100

fig, axes = plt.subplots(1, 2, figsize=(16, 6))
sns.barplot(data=region_plot, x="broad_region", y="events", color="steelblue", ax=axes[0])
axes[0].set_title("Event counts vs. major-quake share")
axes[0].set_xlabel("Broad region")
axes[0].set_ylabel("Events (count)")
axes[0].tick_params(axis="x", rotation=20)
axes[0].grid(True, axis="y", linestyle="--", alpha=0.3)

ax2 = axes[0].twinx()
sns.lineplot(data=region_plot, x="broad_region", y="pct_major", marker="o", color="darkred", ax=ax2)
ax2.set_ylabel("% of events Mw >= 7")
ax2.set_ylim(0, max(5, region_plot["pct_major"].max() + 2))
ax2.grid(False)

if depth_mix.empty:
    axes[1].text(0.5, 0.5, "Depth categories unavailable", ha="center", va="center", fontsize=12)
    axes[1].axis("off")
else:
    sns.heatmap(
        depth_mix,
        annot=True,
        fmt=".1f",
        cmap="YlOrRd",
        cbar_kws={"label": "% per region"},
        ax=axes[1]
    )
    axes[1].set_title("Depth mix by region (% of row)")
    axes[1].set_xlabel("Depth category")
    axes[1].set_ylabel("")

plt.tight_layout()

region_summary = region_summary.round({"median_mag": 2, "iqr_mag": 2, "pct_major": 1, "median_depth": 0})
region_summary


- **Volume vs. severity** - the dual-axis figure surfaces whether high-volume regions also shoulder most Mw ≥ 7 events.
- **Depth mix heatmap** - row-normalised percentages flag shallow-crust dominated regions vs. deep subduction zones.
- **Summary table** - use `iqr_mag` and `median_depth` to articulate variability and tectonic style.


#### 8.4 Quality Score & Uncertainty Analysis
`quality_score` compresses normalised gap/rms/error metrics into a 0-1 scale (1 = best). This subsection surfaces measurement reliability before modelling.


In [ ]:
sns = libs.sns
pd = libs.pd
np = libs.np
plt = libs.plt

quality_base_cols = [
    "quality_score", "gap", "rms", "depthError", "magError", "horizontalError",
    "mag", "mag_category", "broad_region", "time", "place", "id"
]
quality_cols = [c for c in quality_base_cols if c in featured_eq_df.columns]
quality_df = featured_eq_df[quality_cols].dropna(subset=["quality_score"]).copy()
if quality_df.empty:
    raise ValueError("quality_score not available for Section 8.4.")

quantiles = quality_df["quality_score"].quantile([0.25, 0.5, 0.75]).to_dict()

fig, axes = plt.subplots(1, 2, figsize=(16, 6))
sns.histplot(
    data=quality_df,
    x="quality_score",
    bins=30,
    color="slateblue",
    edgecolor="white",
    ax=axes[0]
)
axes[0].set_title("Quality score distribution (1 = best)")
axes[0].set_xlabel("quality_score")
axes[0].set_ylabel("Event count")
for quant, label, color in [
    (0.25, "25th percentile", "#8884d8"),
    (0.50, "Median", "#222222"),
    (0.75, "75th percentile", "#ff7f0e")
]:
    axes[0].axvline(quantiles[quant], color=color, linestyle="--", linewidth=1.2, label=f"{label}: {quantiles[quant]:.2f}")
axes[0].legend(loc="best", frameon=False)
axes[0].grid(True, linestyle="--", alpha=0.3)

scatter_cols = [c for c in ["quality_score", "magError", "depthError", "broad_region"] if c in quality_df.columns]
scatter_df = quality_df[scatter_cols].dropna()
axes[1].set_title("Quality vs. magnitude error")
axes[1].set_xlabel("quality_score")
axes[1].set_ylabel("magError")
if scatter_df.empty:
    axes[1].text(0.5, 0.5, "Missing error columns", ha="center", va="center", fontsize=12)
    axes[1].grid(False)
else:
    hue_col = "broad_region" if "broad_region" in scatter_df.columns else None
    size_col = "depthError" if "depthError" in scatter_df.columns else None
    sns.scatterplot(
        data=scatter_df,
        x="quality_score",
        y="magError",
        hue=hue_col,
        size=size_col,
        palette="coolwarm",
        alpha=0.7,
        ax=axes[1]
    )
    axes[1].grid(True, linestyle="--", alpha=0.3)
    axes[1].invert_xaxis()
    axes[1].legend(loc="best")

plt.tight_layout()

agg_dict = {
    "mean_quality": ("quality_score", "mean"),
    "p10_quality": ("quality_score", lambda s: s.quantile(0.10)),
    "p90_quality": ("quality_score", lambda s: s.quantile(0.90))
}
if "magError" in quality_df.columns:
    agg_dict["median_magError"] = ("magError", "median")
if "depthError" in quality_df.columns:
    agg_dict["median_depthError"] = ("depthError", "median")
if "horizontalError" in quality_df.columns:
    agg_dict["median_horizontalError"] = ("horizontalError", "median")

region_quality = quality_df.groupby("broad_region").agg(**agg_dict)
region_quality.insert(0, "events", quality_df.groupby("broad_region").size())
region_quality = region_quality.sort_values("mean_quality", ascending=False)
region_quality = region_quality.round(3)

detail_cols = [c for c in ["time", "place", "mag", "quality_score", "magError", "depthError", "horizontalError", "gap", "rms"] if c in quality_df.columns]
low_quality_events = quality_df.nsmallest(min(5, len(quality_df)), "quality_score")[detail_cols]

region_quality, low_quality_events


- **Histogram** - cite percentile markers to describe overall measurement fidelity; a tight cluster near 1 indicates robust instrumentation.
- **Scatter** - low-quality (left) observations aligning with high `magError` should be down-weighted or excluded during modelling.
- **Tables** - region-level metrics and the tail of poorest events (IDs, locations, error terms) guide manual curation.


### 9. Modelling
This section outlines how to operationalise the engineered catalogue into a classifier for strong events and an optional ordinal model for magnitude buckets.

#### 9.1 Problem Definition
- Primary objective: flag strong earthquakes (mag >= 6.0) quickly; optimise for recall/precision on the positive class because the catalog is imbalanced.
- Secondary objective: predict `mag_category` as an ordinal multi-class task to prioritise response tiers.
- Split train/validation/test chronologically (e.g., 70/15/15) and stratify by `broad_region` to limit spatial leakage.

#### 9.2 Feature Selection & Preprocessing
- Candidate predictors: depth, latitude/longitude, `gap`, `rms`, `depthError`, `magError`, `horizontalError`, `quality_score`, `depth_category_code`, `broad_region_code`, hemisphere codes, `part_of_day`, `season`, and distance-from-equator/prime-meridian features. Exclude raw `mag`/`mag_category` when they define the target.
- Handle class imbalance with class weights and/or sample weights proportional to `quality_score`; optionally drop events below a `quality_score` threshold.
- Pipeline: `ColumnTransformer` with `StandardScaler` for numeric features and `OneHotEncoder` for categoricals; impute numeric medians and most-frequent categories. Package the whole pipeline so preprocessing and model stay coupled.

#### 9.3 Model Training
- Baselines: `LogisticRegression` (balanced class weights) plus tree ensembles (RandomForest/GradientBoosting) for non-linear interactions; optionally wrap in `CalibratedClassifierCV` for probability quality.
- Cross-validate (e.g., StratifiedKFold=5) with F1 and PR-AUC for the strong class; tune decision thresholds on the validation split to balance missed alerts vs false alarms.
- Persist the fitted pipeline with joblib and record metadata (training window, feature list, metrics) in a small JSON manifest.

#### 9.4 Model Evaluation & Interpretation
- Report confusion matrix, precision/recall/F1, ROC-AUC, and PR-AUC on the hold-out set; slice metrics by `broad_region` and `depth_category` to test generalisation.
- Use permutation importance or SHAP (if available) to confirm drivers (depth, region, uncertainty terms); plot partial dependence for depth and `quality_score`.
- Plot calibration curves and reliability diagrams; monitor drift by re-scoring monthly batches as new catalogue data lands.


### 10. Discussion & Conclusion

#### 10.1 Key Findings
- Global epicentres cluster along subduction belts (notably the Pacific rim), with shallow crustal events dominating counts while intermediate/deep slices still host notable strong events.
- Magnitude vs depth shows modest correlation; high-magnitude events are not confined to the deepest bins, reinforcing the need to model across depth classes.
- Quality-score analysis indicates most events are measured with high fidelity, but a thin low-quality tail (high `magError`/`gap`) should be down-weighted or reviewed before modelling.

#### 10.2 Strengths of the Analytical Pipeline
- Reusable cleaning and feature-engineering functions shared between the notebook and export scripts keep results reproducible.
- Interactive spatial visualisations paired with summary tables make regional patterns and uncertainty interpretable for non-technical readers.
- Uncertainty-aware metrics (quality-score slicing, error columns) are surfaced before any predictive step.

#### 10.3 Limitations
- Catalogue completeness varies by region/time; smaller events are under-reported, biasing magnitude and depth distributions.
- `broad_region` is longitude-based and coarse; no focal mechanisms or tectonic setting variables are included yet.
- Modelling remains a blueprint in this notebook; performance claims await an implemented training run.

#### 10.4 Future Work
- Train and validate the outlined classifier/regressor with temporal cross-validation and calibrated probabilities.
- Enrich geography with plate-boundary shapefiles and proximity features; add geodetic/elevation context where available.
- Automate monthly refreshes (data pull, cleaning, plots, model scoring) to keep the dashboard current.

#### 10.5 Conclusion
The EDA establishes data health, spatial clustering, and uncertainty structure. Next steps are to deploy an uncertainty-aware classifier, monitor it over time, and keep incorporating new catalogue data to refine both the map and the model.


### References
- USGS Earthquake Catalog (https://earthquake.usgs.gov/earthquakes/search/) for event data.
- Python stack: pandas, numpy, seaborn, matplotlib, plotly (interactive maps), scikit-learn (modelling pipeline suggestions).
- Basemaps and tiles: Natural Earth (geo projection) and OpenStreetMap tiles consumed through Plotly mapbox/maplibre.
